In [91]:
###### First attempt to conduct machine learning on text data within the student evals 
## install pkgs 
import sys
import os
# !{sys.executable} -m pip install xgboost==1.7.5 # note: needed since it looks like anaconda installs an earlier version 
# of the package, which is not helpful. 1.7.5 allows for the categorical data of interest to be used. 

# !{sys.executable} -m pip install requests #; this code here can be used to install packages on anaconda/jupyter notebook 
### I believe the below should be installed by default 
import requests # web scraping 
from bs4 import BeautifulSoup # for web scraping 
import itertools # for efficient operation of loops 
import pandas as pd # necessary for reading in, creating, and manipulating data frames 
import csv ## for importing/exporting csvs 
import glob ## for finding files in path
import re
import numpy as np
### THe ML packages 
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# sklearn packages; for machine learning metrics and also text stuff (NLP)
from sklearn import metrics
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

In [92]:
#### read in the data 
rmp_df = pd.read_csv("text_cleaning_data/scored_rmp_data06212023.csv")
rmp_df

,row,quality_of_class,difficulty_of_class,class_code,college,prof_firstname,prof_lastname,comment,out_misrep,out_emo_lang,...,pb_origin,pb_nuero_div,pb_phys_able,pb_pol_affil,complex,constructive,reflective,outrage_agg,personal_attack_agg,prejudice_agg
0,1,2.0,4,POLITSC3500,OHIO STATE UNIVERSITY,ALEX,ACS,"Only graded on 4 assignments (30% Midterm, 30%...",0,0,...,0.0,0,0,0.0,0,0,0,0,0,0.0
1,2,3.0,4,POLITSC3500,OHIO STATE UNIVERSITY,ALEX,ACS,"Final grade is only based on two exams, readin...",0,0,...,0.0,0,0,0.0,1,0,0,0,0,0.0
2,3,4.0,1,POLITSC1100,OHIO STATE UNIVERSITY,ALEX,ACS,Class was super easy. One reading quiz a week ...,0,0,...,0.0,0,0,0.0,0,0,0,0,0,0.0
3,4,4.0,2,POLITSC3115,OHIO STATE UNIVERSITY,ALEX,ACS,"Lecture could be dry at times, but I still lik...",0,0,...,0.0,0,0,0.0,0,0,1,0,0,0.0
4,5,5.0,3,POLITSC3115,OHIO STATE UNIVERSITY,ALEX,ACS,"was an excellent lecturer. Insightful, even h...",0,0,...,0.0,0,0,0.0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,4,4.0,4,SOC410,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,She's very fair. I got away with calling Karl ...,0,0,...,0.0,0,0,0.0,0,0,0,0,0,0.0
1096,5,2.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Professor is cool and all but her accent make...,0,0,...,0.0,0,0,0.0,0,0,0,0,0,0.0
1097,6,1.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,"Definitely not ""the greatest thing since slice...",0,0,...,0.0,0,0,0.0,0,0,0,1,0,0.0
1098,7,2.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,"She's not personal, almost to a harsh point. T...",0,0,...,0.0,0,0,0.0,0,0,0,1,1,0.0


In [93]:
### now clean the text 
regex = r"[\"#\$%&\'\(\)\*\+,-\./:;<=>\?@\[\\\]\^_`{\|}~]" # I removed ! from the beginning 

rmp_df['comment'] = rmp_df['comment'].replace(to_replace =regex, value = '', regex = True)
rmp_df['comment'] = rmp_df['comment'].replace(to_replace ="\d+", value = '', regex = True)

#rmp_df['comment'] = re.sub(regex, '', str(rmp_df['comment']).strip() )
#rmp_df['comment'] =  re.sub("\d+", " ", str(rmp_df['comment'] ))

## reading in the stopwords 
stop_words = stopwords.words('english') # creates a list of English stop words
# wnl = WordNetLemmatizer() # I used lemmatizing instead of stemming
 ## looks like it worked now 

pat = '|'.join([r'\b{}\b'.format(w) for w in stop_words])

rmp_df['comment_edit'] = rmp_df['comment'].str.replace(pat, '') ## good, stop words removed 
rmp_df['comment_edit'] = rmp_df['comment_edit'].str.strip()
### now apply stemming

rmp_df

C:\Users\j-curiel\AppData\Local\Temp\ipykernel_31156\3636061428.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  rmp_df['comment_edit'] = rmp_df['comment'].str.replace(pat, '') ## good, stop words removed


,row,quality_of_class,difficulty_of_class,class_code,college,prof_firstname,prof_lastname,comment,out_misrep,out_emo_lang,...,pb_nuero_div,pb_phys_able,pb_pol_affil,complex,constructive,reflective,outrage_agg,personal_attack_agg,prejudice_agg,comment_edit
0,1,2.0,4,POLITSC3500,OHIO STATE UNIVERSITY,ALEX,ACS,Only graded on assignments Midterm Final R...,0,0,...,0,0,0.0,0,0,0,0,0,0.0,Only graded assignments Midterm Final Rea...
1,2,3.0,4,POLITSC3500,OHIO STATE UNIVERSITY,ALEX,ACS,Final grade is only based on two exams reading...,0,0,...,0,0,0.0,1,0,0,0,0,0.0,Final grade based two exams reading quizzes...
2,3,4.0,1,POLITSC1100,OHIO STATE UNIVERSITY,ALEX,ACS,Class was super easy One reading quiz a week w...,0,0,...,0,0,0.0,0,0,0,0,0,0.0,Class super easy One reading quiz week com...
3,4,4.0,2,POLITSC3115,OHIO STATE UNIVERSITY,ALEX,ACS,Lecture could be dry at times but I still like...,0,0,...,0,0,0.0,0,0,1,0,0,0.0,Lecture could dry times I still liked clas...
4,5,5.0,3,POLITSC3115,OHIO STATE UNIVERSITY,ALEX,ACS,was an excellent lecturer Insightful even han...,0,0,...,0,0,0.0,0,0,0,0,0,0.0,excellent lecturer Insightful even handed wil...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,4,4.0,4,SOC410,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Shes very fair I got away with calling Karl Ma...,0,0,...,0,0,0.0,0,0,0,0,0,0.0,Shes fair I got away calling Karl Marx idio...
1096,5,2.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Professor is cool and all but her accent make...,0,0,...,0,0,0.0,0,0,0,0,0,0.0,Professor cool accent makes hard under...
1097,6,1.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Definitely not the greatest thing since slice ...,0,0,...,0,0,0.0,0,0,0,1,0,0.0,Definitely greatest thing since slice bread ...
1098,7,2.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Shes not personal almost to a harsh point The ...,0,0,...,0,0,0.0,0,0,0,1,1,0.0,Shes personal almost harsh point The class ...


In [94]:
rmp_df['comment_edit'].dtypes

dtype('O')

In [95]:
### now apply stemming to the comments 
from nltk.stem import PorterStemmer 
ps = PorterStemmer() 
#rmp_df_sub = rmp_df.iloc[0:2,:]
#for w in rmp_df_sub.comment_edit: 
#    print(w, " : ", ps.stem(w)) 
#rmp_df['stem'] = rmp_df.comment_edit.apply(ps.stem)
    
rmp_df    

,row,quality_of_class,difficulty_of_class,class_code,college,prof_firstname,prof_lastname,comment,out_misrep,out_emo_lang,...,pb_nuero_div,pb_phys_able,pb_pol_affil,complex,constructive,reflective,outrage_agg,personal_attack_agg,prejudice_agg,comment_edit
0,1,2.0,4,POLITSC3500,OHIO STATE UNIVERSITY,ALEX,ACS,Only graded on assignments Midterm Final R...,0,0,...,0,0,0.0,0,0,0,0,0,0.0,Only graded assignments Midterm Final Rea...
1,2,3.0,4,POLITSC3500,OHIO STATE UNIVERSITY,ALEX,ACS,Final grade is only based on two exams reading...,0,0,...,0,0,0.0,1,0,0,0,0,0.0,Final grade based two exams reading quizzes...
2,3,4.0,1,POLITSC1100,OHIO STATE UNIVERSITY,ALEX,ACS,Class was super easy One reading quiz a week w...,0,0,...,0,0,0.0,0,0,0,0,0,0.0,Class super easy One reading quiz week com...
3,4,4.0,2,POLITSC3115,OHIO STATE UNIVERSITY,ALEX,ACS,Lecture could be dry at times but I still like...,0,0,...,0,0,0.0,0,0,1,0,0,0.0,Lecture could dry times I still liked clas...
4,5,5.0,3,POLITSC3115,OHIO STATE UNIVERSITY,ALEX,ACS,was an excellent lecturer Insightful even han...,0,0,...,0,0,0.0,0,0,0,0,0,0.0,excellent lecturer Insightful even handed wil...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,4,4.0,4,SOC410,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Shes very fair I got away with calling Karl Ma...,0,0,...,0,0,0.0,0,0,0,0,0,0.0,Shes fair I got away calling Karl Marx idio...
1096,5,2.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Professor is cool and all but her accent make...,0,0,...,0,0,0.0,0,0,0,0,0,0.0,Professor cool accent makes hard under...
1097,6,1.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Definitely not the greatest thing since slice ...,0,0,...,0,0,0.0,0,0,0,1,0,0.0,Definitely greatest thing since slice bread ...
1098,7,2.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Shes not personal almost to a harsh point The ...,0,0,...,0,0,0.0,0,0,0,1,1,0.0,Shes personal almost harsh point The class ...


In [96]:
### attempt to lemmatize, thereby shortening words 
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

## lematize here 
text_lemmatized = rmp_df.comment_edit.apply(lemmatize_text)

#rmp_df['comment_edit']

In [97]:
#rmp_df.assign(text_lemmatized=text_lemmatized)
rmp_df['text_lemmatized'] = pd.Series(text_lemmatized)

In [98]:
print(rmp_df['text_lemmatized'])

0       [Only, graded, assignment, Midterm, Final, Rea...
1       [Final, grade, based, two, exam, reading, quiz...
2       [Class, super, easy, One, reading, quiz, week,...
3       [Lecture, could, dry, time, I, still, liked, c...
4       [excellent, lecturer, Insightful, even, handed...
                              ...                        
1095    [Shes, fair, I, got, away, calling, Karl, Marx...
1096    [Professor, cool, accent, make, hard, understa...
1097    [Definitely, greatest, thing, since, slice, br...
1098    [Shes, personal, almost, harsh, point, The, cl...
1099    [unpopular, Sociology, professor, OSU, Marion,...
Name: text_lemmatized, Length: 1100, dtype: object


In [99]:
## Let's move on for now and test these later 
## Now we will want to decrease size of data. For now, let's just go with the misrep column
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
#rmp_df_sub = rmp_df.loc[:,'comment_edit','out_misrep']

### now let's split up the data into train and test sets 
# Split the data; follows the same syntax as box 6 via the commas 
X_train, X_test, y_train, y_test = train_test_split(rmp_df['text_lemmatized'], rmp_df['out_misrep'], random_state=1337,
                                                   stratify=rmp_df['out_misrep'], test_size=0.25)

# vectorizing the sentences
#cv = CountVectorizer(binary = True) # implies that it indicates whether the word is present or not.
#cv.fit(train['review']) # find all the unique words from the training set
#train_x = cv.transform(train['review'])
#test_x = cv.transform(test['review'])
X_test

968    [Awesome, professor, There, test, easy, paper,...
201    [Very, sweet, woman, Extremely, helpful, Decen...
455    [Dr, great, professor, She, really, care, stud...
9      [Aces, clearly, love, information, teaching, c...
52     [Prof, good, lecturer, The, class, typical, po...
                             ...                        
778    [He, seemed, friendlyeager, help, student, ter...
78     [great, guy, best, professor, department, I, t...
814    [Makes, class, interesting, seems, knowledgabl...
71     [I, loved, class, They, enthusiastic, much, ex...
700    [Whoever, say, bad, wrong, Dr, really, good, t...
Name: text_lemmatized, Length: 275, dtype: object

In [100]:
text_lemmatized_all = X_train.astype(str)
text_lemmatized_all

276     ['The', 'assignment', 'two', 'group', 'project...
1032    ['She', 'sweet', 'make', 'class', 'interesting...
782     ['Shes', 'sweet', 'person', 'lecture', 'drag',...
3       ['Lecture', 'could', 'dry', 'time', 'I', 'stil...
156     ['My', 'favorite', 'professor', 'EVER', 'I', '...
                              ...                        
277     ['The', 'lecture', 'pretty', 'boring', 'Dr', '...
235     ['Dr', 'amazing', 'professor!', 'She', 'clear'...
494     ['She', 'one', 'favorite', 'teacher', 'tell', ...
920     ['A', 'mess', 'prof', 'direction', 'lecture', ...
165     ['Dr', 'amazing', 'professor!!', 'I', 'took', ...
Name: text_lemmatized, Length: 825, dtype: object

In [101]:
### let's try to apply the above to the data
from sklearn.feature_extraction.text import TfidfVectorizer
# link to cmd syntax: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
X_train = X_train.astype(str)
def tokens(x): return x.split(',')
tfidf_vect= TfidfVectorizer( tokenizer=tokens 
                            ,use_idf=True
                            , smooth_idf=True
                            , min_df = 1
                            , stop_words = 'english',
                            #, max_features = 10
                            sublinear_tf=False,
                           ngram_range=(2,2),
                           lowercase=False)

tf_idf_matrix = pd.DataFrame(
    tfidf_vect.fit_transform(X_train).toarray(), 
    columns=tfidf_vect.get_feature_names()
)

Xtrain_mat=tfidf_vect.fit_transform(X_train)
Xtrain_mat ### good, this is what we want 

C:\Users\j-curiel\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


<825x15041 sparse matrix of type '<class 'numpy.float64'>'
	with 19782 stored elements in Compressed Sparse Row format>

In [102]:
tf_idf_matrix ### this is a visual representation of what's being produced. Note that it is only for the first row 

,'!!' 'He','!!' 'Ive','!!' 'shes','!' 'Class','!' 'Had','!' 'Hell','!' 'I','!' 'Passionate','!' 'She','!' 'The',...,['shes' 'boring',['shouldnt' 'teaching',['smart' 'Pay',['smart' 'guy',['suck' 'get',['sweet' 'caring',['take' 'next',['talk' 'way',['test' 'pop',['took' 'class'
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
822,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
823,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [103]:
### now do the same for the test set 
X_test = X_test.astype(str)
Xtest_mat=tfidf_vect.fit_transform(X_test)
Xtest_mat

<275x5907 sparse matrix of type '<class 'numpy.float64'>'
	with 6852 stored elements in Compressed Sparse Row format>

In [104]:
### read in ordinal encorder 
import xgboost as xgb
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
### try to do a grid search 

xgb_train = xgb.DMatrix(Xtrain_mat, y_train, enable_categorical=True)
xgb_test = xgb.DMatrix(Xtest_mat, y_test, enable_categorical=True)


#grid_search = GridSearchCV(estimator = pipe, param_grid = param_grid, 
#                          cv = 3, n_jobs = 1, verbose = 0, return_train_score=True)

#hyperparameter fitting ; where it did not work for me ; also broke down here 
#grid_search.fit(X_train, y_train)

In [105]:
# Setting the Parameters of the Model
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

## set parameters 
params = {"objective": "multi:softprob", "tree_method": "gpu_hist", "num_class": 3}
n = 1000

results = xgb.cv(
   params, xgb_train,
   num_boost_round=n,
   nfold=3,
   metrics=["mlogloss", "auc", "merror"], ## here we are telling to look at three metrics, which are 
    # multi-class log loss, area under the ROC curve, and multi-class classification error
) # note that the auc is the flase positive rate (x axis) plotted against the true positive rate (y-axis). Closer to 1, teh 
# better, with 0 being garbage 

#results.keys() # keys() method returns a list of all the keys in a dictionary, and what can be analyzed. Let's take a look


In [106]:
results['test-auc-mean'].max()# at 0.57; not ideal, but not the worst either. Better than random 



0.5493390209430675

In [107]:
misrep_model =xgb.train(params, xgb_train, num_boost_round = n)

In [108]:
###lets see fit 
#misrep_model.fit(xgb_train, xgb_test)
#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV
#model = GridSearchCV(estimator = misrep_model,param_grid = xgb_test)

y_test_pred = misrep_model.predict(xgb_test)
y_test_pred # seems like this defaults to a naive dist when not paired with data it recognizes 

array([[9.6953034e-01, 2.3030056e-02, 7.4396497e-03],
       [9.6953034e-01, 2.3030056e-02, 7.4396497e-03],
       [9.6953034e-01, 2.3030056e-02, 7.4396497e-03],
       [9.6953034e-01, 2.3030056e-02, 7.4396497e-03],
       [9.6953034e-01, 2.3030056e-02, 7.4396497e-03],
       [9.6953034e-01, 2.3030056e-02, 7.4396497e-03],
       [9.6953034e-01, 2.3030056e-02, 7.4396497e-03],
       [9.6953034e-01, 2.3030056e-02, 7.4396497e-03],
       [9.6953034e-01, 2.3030056e-02, 7.4396497e-03],
       [7.9648620e-01, 1.9081961e-01, 1.2694186e-02],
       [9.6953034e-01, 2.3030056e-02, 7.4396497e-03],
       [9.6953034e-01, 2.3030056e-02, 7.4396497e-03],
       [9.6953034e-01, 2.3030056e-02, 7.4396497e-03],
       [9.6953034e-01, 2.3030056e-02, 7.4396497e-03],
       [9.6953034e-01, 2.3030056e-02, 7.4396497e-03],
       [9.6953034e-01, 2.3030056e-02, 7.4396497e-03],
       [9.6953034e-01, 2.3030056e-02, 7.4396497e-03],
       [9.6953034e-01, 2.3030056e-02, 7.4396497e-03],
       [9.9005437e-01, 6.382

In [90]:
### save model 
# save to JSON
misrep_model.save_model("model_06222023.json")

## to load in: 
misrep_model = xgb.Booster()
misrep_model.load_model("model.json")

In [53]:
### let's apply to our test data 
y_pred = misrep_model.predict(xgb_test)
y_pred # this returns an array, which is 3 cols 

array([[9.9572092e-01, 3.3867841e-03, 8.9231879e-04],
       [9.9675626e-01, 1.4976424e-03, 1.7460856e-03],
       [9.9279273e-01, 5.7049468e-03, 1.5022800e-03],
       ...,
       [9.6849906e-01, 2.8263498e-02, 3.2375008e-03],
       [9.8419487e-01, 1.4423719e-02, 1.3813366e-03],
       [9.0871793e-01, 8.8724770e-02, 2.5573066e-03]], dtype=float32)

In [54]:
y_pred_df = pd.DataFrame(y_pred, columns=['not_misrep','somewhat_misrep','very_misrep'])
y_pred_df.head(10)

,not_misrep,somewhat_misrep,very_misrep
0,0.995721,0.003387,0.000892
1,0.996756,0.001498,0.001746
2,0.992793,0.005705,0.001502
3,0.973856,0.011509,0.014635
4,0.983300,0.014799,0.001901
5,0.983300,0.014799,0.001901
6,0.986898,0.011239,0.001862
7,0.996881,0.001522,0.001597
8,0.995054,0.001883,0.003062
9,0.997689,0.001817,0.000494


In [55]:
y_pred_df['not_misrep'].min() # lowest is still 54 %; means most are not considered misrep. 

0.18419794738292694

In [56]:
### let's predict onto the total data set 
x_final = rmp_df['text_lemmatized']

### clean type 
x_final = x_final.astype(str)
x_final_mat=tfidf_vect.fit_transform(x_final)

## check 


x_final_mat = xgb.DMatrix(x_final_mat, enable_categorical=True)
x_final_mat

In [57]:
### now let's try to predict 
post_model_pred = misrep_model.predict(x_final_mat)
post_model_pred ## good 

array([[9.2476314e-01, 1.5898839e-02, 5.9338052e-02],
       [9.0144825e-01, 9.4556957e-02, 3.9948090e-03],
       [9.9688524e-01, 2.5627632e-03, 5.5202644e-04],
       ...,
       [9.1458035e-01, 7.5135946e-02, 1.0283697e-02],
       [9.5847327e-01, 1.9467477e-02, 2.2059243e-02],
       [9.9813062e-01, 7.4268936e-04, 1.1266695e-03]], dtype=float32)

In [58]:
### now create df 
post_model_pred_df = pd.DataFrame(post_model_pred, columns=['not_misrep','somewhat_misrep','very_misrep'])
post_model_pred_df.head(10)

,not_misrep,somewhat_misrep,very_misrep
0,0.924763,0.015899,0.059338
1,0.901448,0.094557,0.003995
2,0.996885,0.002563,0.000552
3,0.996198,0.001970,0.001832
4,0.989664,0.001846,0.008490
5,0.924763,0.015899,0.059338
6,0.996572,0.001341,0.002087
7,0.948045,0.048637,0.003318
8,0.998607,0.000904,0.000489
9,0.992481,0.005857,0.001663


In [59]:
### lets combine the data frame and take a look 
rmp_misrep = rmp_df.iloc[:,1:9]
rmp_misrep2 =pd.concat([rmp_misrep, post_model_pred_df], axis=1)
rmp_misrep2


,quality_of_class,difficulty_of_class,class_code,college,prof_firstname,prof_lastname,comment,out_misrep,not_misrep,somewhat_misrep,very_misrep
0,2.0,4,POLITSC3500,OHIO STATE UNIVERSITY,ALEX,ACS,Only graded on assignments Midterm Final R...,0,0.924763,0.015899,0.059338
1,3.0,4,POLITSC3500,OHIO STATE UNIVERSITY,ALEX,ACS,Final grade is only based on two exams reading...,0,0.901448,0.094557,0.003995
2,4.0,1,POLITSC1100,OHIO STATE UNIVERSITY,ALEX,ACS,Class was super easy One reading quiz a week w...,0,0.996885,0.002563,0.000552
3,4.0,2,POLITSC3115,OHIO STATE UNIVERSITY,ALEX,ACS,Lecture could be dry at times but I still like...,0,0.996198,0.001970,0.001832
4,5.0,3,POLITSC3115,OHIO STATE UNIVERSITY,ALEX,ACS,Alex was an excellent lecturer Insightful even...,0,0.989664,0.001846,0.008490
...,...,...,...,...,...,...,...,...,...,...,...
1095,4.0,4,SOC410,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Shes very fair I got away with calling Karl Ma...,0,0.990735,0.004749,0.004516
1096,2.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Professor klochko is cool and all but her acce...,0,0.918948,0.077825,0.003227
1097,1.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Definitely not the greatest thing since slice ...,0,0.914580,0.075136,0.010284
1098,2.0,4,SOC101,OHIO STATE UNIVERSITY,MARIANNA,KLOCHKO,Shes not personal almost to a harsh point The ...,0,0.958473,0.019467,0.022059


In [148]:
## write out csv 
rmp_misrep2.to_csv("output/misrep_predicted_06212023.csv")

In [160]:
### convert pred probs to class 
idx = np.argmax(y_pred, axis=-1)
y_pred = np.zeros( y_pred.shape )
y_pred[ np.arange(y_pred.shape[0]), idx] = 1

y_pred ## seems to have worked; now we just need to get to long 

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [168]:
## create data frame 

y_pred_df = pd.DataFrame(y_pred, columns=['not_misrep','somewhat_misrep','very_misrep'])
### now create a column based on max 
y_pred_df['label'] = np.where(y_pred_df['not_misrep'] == 1, 0, 1)
y_pred_df.loc[y_pred_df.somewhat_misrep==1, 'label'] == 1
y_pred_df.loc[y_pred_df.very_misrep==1, 'label'] == 2
y_pred_df['label'].unique() ## none of these are above 0. This implies issues wrt too many non misrep comments. This might 
# not be too big of an issue, assuming that we can get all of these dimensions summed up 


array([0])

In [161]:
### let's look at accuracy 
xgb_test_labels = []
for rating in y_test:
    if rating == 0:
        xgb_test_labels.append(0)
    elif rating == 2:
        xgb_test_labels.append(2)
    else:
        xgb_test_labels.append(1)
f1_score(xgb_test_labels, y_pred) # supposed to be  94.83%

ValueError: Classification metrics can't handle a mix of multiclass and multilabel-indicator targets

In [151]:
y_pred

array([[9.5794606e-01, 3.6805231e-02, 5.2486709e-03],
       [9.9901891e-01, 6.8283116e-04, 2.9829287e-04],
       [9.5847327e-01, 1.9467477e-02, 2.2059243e-02],
       ...,
       [9.8856038e-01, 4.9981573e-03, 6.4414935e-03],
       [9.7794914e-01, 2.0888600e-02, 1.1622668e-03],
       [9.9745852e-01, 1.8508084e-03, 6.9073279e-04]], dtype=float32)

In [152]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets

In [154]:
results['test-auc-mean'].max()

0.051551352248216366

In [155]:
import numpy as np  
from sklearn.model_selection import train_test_split  
from sklearn.metrics import accuracy_score   
from sklearn.svm import SVC  
from sklearn.datasets import load_iris  
  
# Loading the dataset  
X, Y = load_iris(return_X_y = True)  
  
# Splitting the dataset in training and test data  
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)  
  
# Training the model using the Support Vector Classification class of sklearn  
svc = SVC()  
svc.fit(X_train, Y_train)  
  
# Computing the accuracy_score of the model  
Y_pred = svc.predict(X_test)  
score = accuracy_score(Y_test, Y_pred)  
print(score)  

0.9777777777777777


In [157]:
Y_pred

array([2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1,
       0, 0, 2, 0, 0, 1, 1, 0, 2, 1, 0, 2, 2, 1, 0, 2, 1, 1, 2, 0, 2, 0,
       0])

In [169]:
rmp_df['text_lemmatized']

0       [Only, graded, assignment, Midterm, Final, Rea...
1       [Final, grade, based, two, exam, reading, quiz...
2       [Class, super, easy, One, reading, quiz, week,...
3       [Lecture, could, dry, time, I, still, liked, c...
4       [Alex, excellent, lecturer, Insightful, even, ...
                              ...                        
1095    [Shes, fair, I, got, away, calling, Karl, Marx...
1096    [Professor, klochko, cool, accent, make, hard,...
1097    [Definitely, greatest, thing, since, slice, br...
1098    [Shes, personal, almost, harsh, point, The, cl...
1099    [Klochko, unpopular, Sociology, professor, OSU...
Name: text_lemmatized, Length: 1100, dtype: object

In [182]:
### now that we are done, lets try and redo a step we tried earlier, namely the tokenizer and tfdf cmd 
from sklearn.feature_extraction.text import TfidfVectorizer
class NewTfidfVectorizer(TfidfVectorizer):
    def _word_ngrams(self, tokens, stop_words=None):

        # First get tokens without stop words
        tokens = super(TfidfVectorizer, self)._word_ngrams(tokens, None)
        if stop_words is not None:
            new_tokens=[]
            for token in tokens:
                split_words = token.split(' ')

                # Only check the first and last word for stop words
                if split_words[0] not in stop_words and split_words[-1] not in stop_words:
                    new_tokens.append(token)
            return new_tokens

        return tokens
    
vectorizer = NewTfidfVectorizer(stop_words='english', ngram_range=(2,3))
# vectorizer.fit(data)

In [183]:
text_lemmatized_all = X_train.astype(str)
text_lemmatized_all

1043    ['Loved', 'Jungers!', 'She', 'made', 'everythi...
122     ['The', 'test', 'class', 'ridiculous', 'made',...
470     ['If', 'psychology', 'major', 'NO', 'point', '...
765     ['midterm', 'final', 'group', 'presentation', ...
2       ['Class', 'super', 'easy', 'One', 'reading', '...
                              ...                        
62      ['Really', 'great', 'guy', 'great', 'class', '...
868     ['Professor', 'Harris', 'nice', 'I', 'found', ...
36      ['Genuinely', 'interested', 'teaching', 'A', '...
342     ['I', 'dont', 'know', 'class', 'psych', 'woman...
529     ['Dr', 'Cudeck', 'brilliant', 'psychologistThe...
Name: text_lemmatized, Length: 737, dtype: object

In [184]:
## now, try the above cmd on the code from earlier regarding the training set 
#tf_idf_matrix = pd.DataFrame(
#    tfidf_vect.fit_transform(X_train).toarray(), 
#    columns=tfidf_vect.get_feature_names()
#)
Xtrain_mat2 =vectorizer.fit_transform(text_lemmatized_all)
Xtrain_mat2 ### good, this is what we want; now we can try some things  

<737x19455 sparse matrix of type '<class 'numpy.float64'>'
	with 22511 stored elements in Compressed Sparse Row format>

In [186]:
tf_idf_matrix_ngramplus = pd.DataFrame(
    vectorizer.fit_transform(text_lemmatized_all).toarray(), 
    columns=vectorizer.get_feature_names()
)

tf_idf_matrix_ngramplus ## good, this demonstrates what I need. We can integrate this into the next version 

C:\Users\j-curiel\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,ab impossible,abbreviated recorded,abbreviated recorded lecture,abcd test,abcd test average,ability clarify,ability he best,ability mainly,able pas,able pas class,...,zero curve,zero curve point,zero quiz,zero quiz test,zone bring,zone bring snack,zoom class,zoom class super,zoom got,zoom got good
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
733,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
734,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
735,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
